## Analyse bornes de recharges publiques pour voitures électriques
source:
https://www.data.gouv.fr/fr/datasets/bornes-de-chargement-publiques-pour-voitures-electriques-du-plusieurs-operateurs/

In [1]:
import requests
import pandas as pd
import numpy as np
import folium
import leafmap
import re
import geopandas as gpd

pd.set_option('display.max_columns', None)

In [2]:
#https://www.data.gouv.fr/fr/datasets/bornes-de-chargement-publiques-pour-voitures-electriques-du-plusieurs-operateurs/
url = "https://www.data.gouv.fr/fr/datasets/r/b11113db-875d-41c7-8673-0cf8ad43e917"
nom_fichier = "Src_bornes_gouv.csv"

response = requests.get(url)

if response.status_code == 200:
    with open(nom_fichier, 'wb') as fichier:
        fichier.write(response.content)
    print("Téléchargement réussi.")
else:
    print("Échec du téléchargement. Code de statut :", response.status_code)

Téléchargement réussi.


In [3]:

#Carte des dépaterments    
url2 = "https://france-geojson.gregoiredavid.fr/repo/departements.geojson"
nom_fichier = "Src_depart_gouv.geojson"

response = requests.get(url2)

if response.status_code == 200:
    with open(nom_fichier, 'wb') as fichier:
        fichier.write(response.content)
    print("Téléchargement réussi.")
else:
    print("Échec du téléchargement. Code de statut :", response.status_code)

Téléchargement réussi.


In [4]:
data_dpt=gpd.read_file('Src_depart_gouv.geojson')

In [5]:
df = pd.read_csv('Src_bornes_gouv.csv')

df.sample(10)

,contact_amenageur,contact_operateur,nom_amenageur,nom_operateur,telephone_operateur,siren_amenageur,raccordement,nom_enseigne,id_station_itinerance,id_station_local,nom_station,implantation_station,adresse_station,coordonneesXY,nbre_pdc,id_pdc_itinerance,id_pdc_local,puissance_nominale,prise_type_ef,prise_type_2,prise_type_combo_ccs,prise_type_chademo,prise_type_autre,gratuit,tarification,paiement_autre,paiement_acte,paiement_cb,condition_acces,reservation,horaires,accessibilite_pmr,restriction_gabarit,station_deux_roues,date_mise_en_service,observations,date_maj,cable_t2_attache
7995,info@chargeguru.com,info@chargeguru.com,NaN,NaN,+33176341178,NaN,NaN,Renault Balesi - 20620 - 3,FRCG0E00022,FRCG0E00022,Renault Balesi - 20620 - 3,Voirie,"Rond-Point de CEPPE, 20620 Biguglia","[42.62879800,9.43704100]",6,FRCG0E000224,FRCG0E000224,22,False,True,False,False,False,False,NaN,NaN,True,False,Accès réservé,True,24/7,Accessibilité inconnue,inconnu,False,2022-08-27,NaN,2023-10-06,False
3377,info@ionity.eu,info@ionity.eu,NaN,NaN,+(33)-(1)-87210891,NaN,NaN,Pech Montat Est,FRIOYE40870,FRIOYE40870,Pech Montat Est,Station dédiée à la recharge rapide,"Aire de service de Pech Montat Est, A20, 46600...","[45.03087800,1.52509800]",5,FRIOYE408704,FRIOYE408704,350,False,False,True,False,False,False,NaN,NaN,True,True,Accès libre,False,24/7,Accessibilité inconnue,inconnu,False,2021-07-14,NaN,2023-10-05,False
3107,klantenservice@newmotion.com,klantenservice@newmotion.com,NaN,NaN,+(33)-(9)-69366018,NaN,NaN,Shell- France Aire de Sommesous HW,FRSHEE45,FRSHEE45,Shell- France Aire de Sommesous HW,Station dédiée à la recharge rapide,"A26 Aire de Sommesous, 51320 Sommesous","[48.72935360,4.22882330]",9,FRSHEE453,FRSHEE453,300,False,False,True,False,False,False,NaN,NaN,True,False,Accès libre,False,24/7,Accessibilité inconnue,inconnu,False,2023-04-08,NaN,2023-10-05,False
10296,support@fastned.nl,support@fastned.nl,Fastned France,NaN,+(33) 184710062,853300010.0,Direct,Aire d’Écot Sud,FRFASE330270,FRFASE330270,Aire d’Écot Sud,Station dédiée à la recharge rapide,"A36, 25150 Écot","[47.44048231,6.72891209]",4,FRFASE3302701,FRFASE3302701,300,False,False,True,False,False,False,0.59€ / kwh,True,True,False,Accès libre,False,24/7,Accessible mais non réservé PMR,Hauteur maximale 3m,False,2021-11-30,NaN,2023-10-05,False
4534,info@chargepoint.com,info@chargepoint.com,NaN,NaN,NaN,NaN,NaN,RAW PARISGIVERNY 67,FRCPIE6736995,FRCPIE6736995,RAW PARISGIVERNY 67,Voirie,"Avenue du Capitaine Vandière de Vitrac Douain,...","[49.06076200,1.43122200]",10,FRCPIE67369951,FRCPIE67369951,21,False,True,False,False,False,False,NaN,NaN,True,False,Accès réservé,False,24/7,Accessibilité inconnue,inconnu,False,2023-05-13,EF connector is available at the site separately,2023-10-05,False
4972,info@allego.eu,info@allego.eu,NaN,NaN,+(33)-(800)-78192,NaN,NaN,Allego Carrefour La Chapelle Saint Luc,FRALLEGO900523,FRALLEGO900523,Allego Carrefour La Chapelle Saint Luc,Voirie,"4 Boulevard de l'Ouest, 10600 La Chapelle Sain...","[48.30589000,4.02569000]",8,FRALLEGO9005232,FRALLEGO9005232,22,False,True,False,False,False,False,NaN,NaN,True,True,Accès libre,False,24/7,Accessibilité inconnue,inconnu,False,2022-04-07,NaN,2023-10-05,False
2059,info@chargepoint.com,info@chargepoint.com,NaN,NaN,NaN,NaN,NaN,CGED CARCASSONNE CGE,FRCPIE6641115,FRCPIE6641115,CGED CARCASSONNE CGE,Voirie,"11 Rte de Bram, 11000 Carcassonne","[43.21449300,2.32292400]",2,FRCPIE66411151,FRCPIE66411151,21,False,True,False,False,False,False,NaN,NaN,True,False,Accès libre,False,"Mo 06:30-19:00,Tu 06:30-19:00,We 06:30-19:00,T...",Accessibilité inconnue,inconnu,False,2022-10-17,EF connector is available at the site separately,2023-10-05,False
10337,kontakt@lidl.de,kontakt@lidl.de,NaN,NaN,+(33)-(800)-900343,NaN,NaN,St Barthelemy Beaurepaire,FRLDLE0000152,FRLDLE0000152,St Barthelemy Beaurepaire,Voirie,"Route de Beaurepaire, 38270 Saint-Barthelemy","[45.34276100,5.06569000]",2,FRLDLE00001520,FRLDLE00001520,22,False,True,False,False,False,False,NaN,

In [6]:
df[['x', 'y']] = df['coordonneesXY'].str.strip('[]').str.split(',', expand=True).astype(float)

def extraire_cp(texte):
    match = re.search(r'\d{5}', texte)
    if match:
        chiffres = match.group()
        return chiffres[:2]  # Prendre les deux premiers chiffres
    else:
        return None

# Appliquez la fonction à la colonne 'Adresse' du DataFrame
df['Departement'] = df['adresse_station'].apply(extraire_cp)



In [7]:
df_region = pd.read_csv('POP_donnees_communes.csv',sep=";")
df_region=df_region[['REG', 'CODDEP',  'COM', 'PTOT']]
df_region = df_region.groupby('CODDEP').sum().reset_index()
df_region

,CODDEP,PTOT
0,01,668565
1,02,543368
2,03,344922
3,04,168867
4,05,145791
...,...,...
95,95,1259985
96,971,389995
97,972,369406
98,973,284085


In [8]:
data_dpt

,code,nom,geometry
0,02,Aisne,"POLYGON ((3.17270 50.01200, 3.18220 50.01234, ..."
1,10,Aube,"POLYGON ((3.41479 48.39027, 3.42208 48.41334, ..."
2,14,Calvados,"POLYGON ((-1.11962 49.35557, -1.11503 49.36240..."
3,15,Cantal,"POLYGON ((2.50841 45.47850, 2.52444 45.48070, ..."
4,28,Eure-et-Loir,"POLYGON ((0.81482 48.67016, 0.82767 48.68072, ..."
...,...,...,...
91,41,Loir-et-Cher,"POLYGON ((0.84122 48.10306, 0.87588 48.10943, ..."
92,53,Mayenne,"POLYGON ((-0.86036 48.50146, -0.84611 48.49828..."
93,54,Meurthe-et-Moselle,"MULTIPOLYGON (((5.45556 49.48093, 5.44953 49.4..."
94,79,Deux-Sèvres,"POLYGON ((-0.89196 46.97582, -0.87973 46.97580..."


In [9]:
df_count_stat = df.groupby('Departement')['adresse_station'].count()
df_region = pd.merge(df_region,df_count_stat,left_on='CODDEP',right_on='Departement')
df_region['Nb stat pour 100khab'] = round((df_region['adresse_station']*100000)/df_region['PTOT'])
df_region.rename(columns={'PTOT': 'Population'}, inplace=True)

In [10]:
data_dpt=pd.merge(data_dpt,df_region,left_on='code',right_on='CODDEP',suffixes=False)
data_dpt


,code,nom,geometry,CODDEP,Population,adresse_station,Nb stat pour 100khab
0,02,Aisne,"POLYGON ((3.17270 50.01200, 3.18220 50.01234, ...",02,543368,71,13.0
1,10,Aube,"POLYGON ((3.41479 48.39027, 3.42208 48.41334, ...",10,317366,58,18.0
2,14,Calvados,"POLYGON ((-1.11962 49.35557, -1.11503 49.36240...",14,709064,140,20.0
3,15,Cantal,"POLYGON ((2.50841 45.47850, 2.52444 45.48070, ...",15,149664,13,9.0
4,28,Eure-et-Loir,"POLYGON ((0.81482 48.67016, 0.82767 48.68072, ...",28,441931,234,53.0
...,...,...,...,...,...,...,...
89,41,Loir-et-Cher,"POLYGON ((0.84122 48.10306, 0.87588 48.10943, ...",41,337975,283,84.0
90,53,Mayenne,"POLYGON ((-0.86036 48.50146, -0.84611 48.49828...",53,316615,49,15.0
91,54,Meurthe-et-Moselle,"MULTIPOLYGON (((5.45556 49.48093, 5.44953 49.4...",54,745453,126,17.0
92,79,Deux-Sèvres,"POLYGON ((-0.89196 46.97582, -0.87973 46.97580...",79,385150,90,23.0


In [11]:
data_dpt.to_file("Src_dpt_final.geojson", driver='GeoJSON')

In [12]:
df_map = df[['nom_enseigne','nom_station', 'implantation_station',
       'adresse_station','nbre_pdc', 'puissance_nominale','gratuit', 'tarification',
       'paiement_cb','restriction_gabarit', 'station_deux_roues',
       'date_mise_en_service', 'observations', 'date_maj',
       'x', 'y', 'Departement']]

In [13]:


ma_carte = leafmap.Map(center=[45.76077534099906, 4.833680068867414],zoom=12)

ma_carte.add_data('Src_dpt_final.geojson', column='Nb stat pour 100khab' ,scheme='Quantiles',legend_title="Bornes pour 100khab", cmap='Blues',layer_name='Departements')
        
ma_carte.add_points_from_xy(data=df_map, x='y', y='x', radius=10)
#ma_carte.add_data(data=df_region, column='Nb stat pour 100khab', scheme='Quantiles', cmap='Blues', legend_title='Population'
#) 


In [14]:
ma_carte

Map(center=[45.76077534099906, 4.833680068867414], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [18]:
ma_carte.to_html("output.html")

In [16]:
df_region.sort_values(by='Nb stat pour 100khab')

,CODDEP,Population,adresse_station,Nb stat pour 100khab
90,92,1640721,41,2.0
91,93,1651739,48,3.0
73,75,2182174,59,3.0
63,65,235709,9,4.0
92,94,1415845,73,5.0
...,...,...,...,...
26,28,441931,234,53.0
53,55,189535,144,76.0
17,18,309523,250,81.0
39,41,337975,283,84.0
